In [ ]:
!pip install datasets transformers evaluate rouge_score torch accelerate langchain-google-genai python-dotenv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━

In [ ]:
import os
import time
# from dotenv import load_dotenv # No longer needed if using Colab Secrets
import google.generativeai as genai
import datasets
import evaluate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata # Import userdata

def generate_summary_with_gemini(prompt_text: str) -> str:
    """주어진 프롬프트 텍스트를 기반으로 Gemini 모델을 사용해 요약문을 생성합니다."""
    try:
        # 사용할 모델 설정
        model = genai.GenerativeModel('gemini-1.5-flash')

        # 모델에 콘텐츠 생성 요청
        response = model.generate_content(prompt_text)

        # 결과 텍스트 반환
        return response.text
    except Exception as e:
        # API 호출 중 에러 발생 시
        return f"요약 생성 중 오류가 발생했습니다: {e}"

In [ ]:
def main():
    """메인 실행 함수: API 설정, 프롬프트 정의, 요약 생성 및 결과 출력"""
    # ⚠️ 중요: 이곳에 발급받은 본인의 API 키를 입력하세요.
    print("🔑 Google API 키를 로드합니다...")
    # Load API key from Colab Secrets
    google_api_key = userdata.get('GOOGLE_API_KEY')

    if not google_api_key:
        print("API 키가 설정되지 않았습니다. Colab의 Secrets Manager에 'GOOGLE_API_KEY'를 설정해주세요.")
        return

    # Configure genai with the loaded API key
    genai.configure(api_key=google_api_key)


    # 1. 프롬프트 설계
    prompt_1 = {
        "name": "뉴스 에디터' (간결하고 핵심적인 요약)",
        "text": """
당신은 뉴스 기사에서 핵심 사실만 추출하는 '정보 요약가'입니다. 다음 뉴스 기사의 제목과 본문을 읽고, 아래 조건에 따라 핵심 내용을 3~4개의 불렛포인트로 요약해 주세요.

[요약 조건]
- 기사에 명시된 객관적인 사실만 포함하세요.
- 추측, 의견, 감정적인 표현은 절대 사용하지 마세요.
- 각 문장은 핵심 내용을 담아 간결하게 작성하세요.
- 독자가 기사의 핵심 팩트를 30초 안에 파악할 수 있어야 합니다.
"""
    }

    prompt_2 = {
        "name": "정보 큐레이터(정보를 흥미롭게 재구성)",
        "text": """
당신은 독자들의 호기심을 자극하는 정보 큐레이터입니다. 다음 뉴스 기사에서 놓치지 말아야 할 핵심 포인트를 발췌하여 독자가 흥미를 느끼고 더 궁금해할 만한 요약문을 작성해주세요.

다음 조건에 따라 요약하세요:
- 각 뉴스 기사의 제목과 함께, 가장 중요한 사실 3~4가지를 압축하여 요약하세요.
- 단순히 요약하는 것을 넘어, 독자의 흥미를 유발할 수 있는 매력적인 표현을 사용해보세요. (단, 과장되거나 허위 사실 포함 금지)
- 요약 내용은 기사 본문의 사실에 기반하되, 독자에게 '왜 이 뉴스를 알아야 하는가'에 대한 이유를 간접적으로 제시해 주세요.
- 각 요약은 짧은 문단 또는 핵심 문장 위주의 불릿 포인트 형식으로 구성할 수 있습니다.

출력 형식:

---

### 📰 {뉴스 제목}
- [핵심 포인트 1]
- [핵심 포인트 2]
- [핵심 포인트 3]
- [핵심 포인트 4 (필요 시)]

---
"""
    }

    prompt_3 = {
        "name": "사용자 맞춤형 뉴스 브리퍼' (핵심 정보와 간략한 배경 제공)",
        "text": """
당신은 바쁜 사용자를 위한 맞춤형 뉴스 브리퍼입니다. 다음 뉴스 기사의 주요 내용을 신속하게 전달하고, 사용자가 이 뉴스를 이해하는 데 필요한 핵심 정보를 간략히 제공해주세요.

요약 가이드라인:
- 각 뉴스 제목 아래에 핵심 내용을 3~4문장으로 요약합니다.
- 요약문에는 기사의 핵심 사실과 더불어, 해당 내용이 가지는 의미나 배경을 간략히 언급하여 사용자의 이해를 돕습니다. (단, 해석이나 의견은 지양하고 사실 기반으로만 설명)
- 핵심 키워드를 포함하여 정보 전달력을 높여주세요.
- 문장은 명료하고 전문적인 톤으로 작성하세요.

출력 형식:

---

### 📰 {뉴스 제목}
{요약문}

---

"""
    }
    prompts = [prompt_1, prompt_2, prompt_3]

    # 2. 모델 입력 구성 (뉴스 기사 정의)
    news_title = """한은 '화폐 위조범 10대까지 확산…SNS 통한 판매 늘어'"""
    news_document = """
    (서울=연합뉴스) 민선희 기자 = 한국은행은 화폐 위조범 연령대가 10대 청소년 등 젊은 층까지 확산하고, 사회관계망서비스(SNS) 등을 통한 위조지폐 판매도 늘어나고 있다고 18일 밝혔다.

한은에 따르면 위폐 방지 실무위원회는 지난 17일 오전 한은 본관에서 2025년 상반기 정기회의를 열었다.

회의 참석자들은 위조지폐 발견 건수가 감소세지만, 최근 위조지폐 제조·유통 양상이 달라지고 있어 경각심을 유지할 필요가 있다고 했다.

과거에는 화폐 위조범 연령대가 주로 고령층이었으나 최근 고성능 컬러프린터의 보편화로 10대 등 젊은 층까지 확산하고 있으며, 위조지폐 유통 채널도 SNS 등 온라인 공간으로 확대됐다.

또한 과거에는 위조지폐가 주로 고령층이 운영하는 전통시장, 노점에서 사용됐다면 최근에는 편의점, 복권판매점, 택시, 식당, 주유소 등 대면 시간이 짧은 업종의 혼잡한 시간대를 노린 사례도 발견됐다.

최근 미국 달러화를 중심으로 국내에서 발견되는 외화 위조지폐 건수와 금액도 늘어나는 추세다.

관세청은 지난 5월 부산항에 정박 중인 국제무역선 선원이 소지하고 있던 위조지폐 미화 100달러권을 적발하기도 했다.

한은은 위조지폐 제조·유통을 효과적으로 차단하기 위해 유관기관 간 긴밀한 공조 체제를 유지하는 한편, 건전한 화폐 유통 질서가 확립될 수 있도록 일반 국민 대상 위폐 유통 방지 홍보를 지속한다는 계획이다.
    """
    full_news_text = f"제목: {news_title}\n본문: {news_document}"

    # 3. 프롬프트별 요약 생성 및 비교 출력
    print("=" * 80)
    print("동일 뉴스 기사에 대한 프롬프트별 요약 결과 비교")
    print(f"사용 모델: gemini-1.5-flash")
    print("=" * 80)

    for i, p_info in enumerate(prompts):
        print(f"\n--- [ 프롬프트 #{i+1}: {p_info['name']} ] ---\n")

        # 프롬프트와 뉴스 기사 결합
        input_text = f"{p_info['text']}\n\n[뉴스 기사]\n{full_news_text}"

        # 요약 생성
        summary_result = generate_summary_with_gemini(input_text)

        # 결과 출력
        print("✨ 생성된 요약문:")
        print(summary_result)
        print("-" * 80)


In [ ]:
if __name__ == '__main__':
    main()

🔑 Google API 키를 로드합니다...
동일 뉴스 기사에 대한 프롬프트별 요약 결과 비교
사용 모델: gemini-1.5-flash

--- [ 프롬프트 #1: 뉴스 에디터' (간결하고 핵심적인 요약) ] ---

✨ 생성된 요약문:
* 한국은행은 화폐 위조범의 연령대가 10대 청소년 등 젊은 층으로 확산되고 있다고 밝혔다.
* 위조지폐 유통 채널이 SNS 등 온라인으로 확대되었으며, 편의점, 택시 등 대면 시간이 짧은 업종을 노리는 사례가 증가했다.
* 위조지폐 발견 건수는 감소세이나, 위조지폐 제조 및 유통 방식의 변화에 대한 경각심이 필요하다.
* 최근 미국 달러화 위조지폐 발견 건수 및 금액 또한 증가하고 있다.

--------------------------------------------------------------------------------

--- [ 프롬프트 #2: 정보 큐레이터(정보를 흥미롭게 재구성) ] ---

✨ 생성된 요약문:
---

### 📰 한은 '화폐 위조범 10대까지 확산…SNS 통한 판매 늘어'

- **10대 위조범 등장: 놀라운 변화!**: 과거 고령층 중심이었던 화폐 위조범들이 10대 청소년까지 확대되면서,  새로운 위협의 시대가 열렸습니다. 고성능 프린터의 보급이 이러한 변화의 원인으로 지목되고 있습니다.  내 지갑 속 돈이 진짜인지 의심하게 만드는 현실, 여러분은 안전하신가요?

- **SNS 판매 확산: 익명성의 그림자**:  위조지폐 유통 채널이 온라인으로 확장되면서,  더욱 은밀하고 광범위하게 위조지폐가 유통될 위험이 커지고 있습니다.  SNS의 익명성이 위조범들에게 완벽한 은신처를 제공하고 있는 셈입니다.

- **유통 방식의 변화: 짧고 굵게!**: 과거 전통시장 등에서 주로 사용되던 위조지폐가 이제는 편의점, 택시 등 대면 시간이 짧은 곳을 노리고 있습니다.  범죄자들의 교활한 수법에 더욱 주의해야 할 이유입니다.

- **외화 위조 증가: 국제적 문제로 확대**:  미국 달러화를 중심으로